<a href="https://colab.research.google.com/github/rvkushnir/project_fifa_players/blob/main/02_analytical_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

План аналітичного блоку.
1. Побудуємо ТОП-100 за overall.
2. Побудуємо ТОП-100 за wage_uer, порівняємо з 1: перетин, розбіжності, Jaccard.
3. ТОП-30 воротарів за overall.
4. ТОП-30 клубів за середнім overall.
5. ТОП-30 клубів за середньою швидкістю (використаємо movement_sprint_speed, або pace як fallback).
6. Ліги з найвищим середнім dribbling (+ розмір вибірки).
7. Рейтинг клубів за стартовим складом 11 гравців (1 GK, 4  DEF, 4 MID, 2 FWD) на основі primary_position.


In [ ]:
#  Завантаження з кешу, якщо є
from pathlib import Path
import pandas as pd
import numpy as np

PARQUET_PATH = Path("data/interim/players_clean.parquet")
DATA_CSV = Path("data/raw/fifa_players.csv")
OUT = Path("out/tables"); OUT.mkdir(parents=True, exist_ok=True)

def make_primary_position(s: pd.Series) -> pd.Series:
    # Беремо перший запис з player_positions як primary
    return s.astype(str).str.split(",", n=1, expand=True)[0].str.strip()

if PARQUET_PATH.exists():
    df = pd.read_parquet(PARQUET_PATH)
else:
    df = pd.read_csv(DATA_CSV, low_memory=False)
    df["primary_position"] = make_primary_position(df["player_positions"])

#  Базові колонки для зручності виводу
base_cols = [c for c in [
    "sofifa_id","short_name","long_name","age","overall","potential",
    "value_eur","wage_eur","club_name","league_name","nationality_name","primary_position"
] if c in df.columns]

# (1) Топ-100 за рейтингом
top100_overall = (
    df.sort_values("overall", ascending=False)
      .loc[:, base_cols].head(100)
)
top100_overall.to_csv(OUT/"top100_overall.csv", index=False)

# (2) Топ-100 за зарплатою + порівняння з (1)
top100_wage = (
    df.sort_values("wage_eur", ascending=False)
      .loc[:, base_cols].head(100)
)
top100_wage.to_csv(OUT/"top100_wage.csv", index=False)

set_overall = set(top100_overall["sofifa_id"])
set_wage = set(top100_wage["sofifa_id"])
intersection_ids = set_overall & set_wage
only_overall_ids = set_overall - set_wage
only_wage_ids    = set_wage - set_overall
jaccard = len(intersection_ids) / len(set_overall | set_wage)

pd.DataFrame([
    {"metric":"count_top100_overall", "value": len(set_overall)},
    {"metric":"count_top100_wage",    "value": len(set_wage)},
    {"metric":"intersection",         "value": len(intersection_ids)},
    {"metric":"jaccard_index",        "value": round(jaccard, 4)}
]).to_csv(OUT/"compare_top100_summary.csv", index=False)

df[df["sofifa_id"].isin(only_overall_ids)][base_cols].to_csv(OUT/"top100_only_overall.csv", index=False)
df[df["sofifa_id"].isin(only_wage_ids)][base_cols].to_csv(OUT/"top100_only_wage.csv", index=False)
df[df["sofifa_id"].isin(intersection_ids)][base_cols].to_csv(OUT/"top100_intersection.csv", index=False)

# (3) Топ-30 воротарів за рейтингом
gk = df[df["primary_position"]=="GK"].copy()
top30_gk = gk.sort_values("overall", ascending=False).loc[:, base_cols].head(30)
top30_gk.to_csv(OUT/"top30_goalkeepers_overall.csv", index=False)

# (4) Топ-30 клубів за середнім рейтингом (з/без порогу 11 гравців)
club_agg = (
    df.groupby("club_name", as_index=False)
      .agg(avg_overall=("overall","mean"),
           n_players=("sofifa_id","nunique"))
      .sort_values(["avg_overall","n_players"], ascending=[False, False])
)
club_agg.to_csv(OUT/"clubs_avg_overall_all.csv", index=False)
club_agg[club_agg["n_players"]>=11].head(30).to_csv(OUT/"clubs_avg_overall_top30_min11.csv", index=False)

# (5) Топ-30 клубів за середньою швидкістю
speed_col = "movement_sprint_speed" if "movement_sprint_speed" in df.columns else ("pace" if "pace" in df.columns else None)
if speed_col is None:
    raise ValueError("Не знайдено стовпців швидкості (movement_sprint_speed чи pace).")
club_speed = (
    df.groupby("club_name", as_index=False)
      .agg(avg_speed=(speed_col,"mean"),
           n_players=("sofifa_id","nunique"))
      .sort_values(["avg_speed","n_players"], ascending=[False, False])
)
club_speed.to_csv(OUT/"clubs_avg_speed_all.csv", index=False)
club_speed[club_speed["n_players"]>=11].head(30).to_csv(OUT/"clubs_avg_speed_top30_min11.csv", index=False)

# (6) Ліги з найкращим дриблінгом
if "dribbling" not in df.columns:
    raise ValueError("Не знайдено стовпця dribbling.")
league_drib = (
    df.groupby("league_name", as_index=False)
      .agg(avg_dribbling=("dribbling","mean"),
           n_players=("sofifa_id","nunique"))
      .sort_values(["avg_dribbling","n_players"], ascending=[False, False])
)
league_drib.to_csv(OUT/"leagues_avg_dribbling_all.csv", index=False)

# (7) Клубний рейтинг за 'стартовим складом 11'
from scipy.optimize import linear_sum_assignment

DEF = {"CB","LB","RB","LWB","RWB"}
MID = {"CDM","CM","CAM","LM","RM"}
FWD = {"ST","CF","LW","RW"}

FLEXIBLE_POSITIONS = True      # увімк/вимк гнучкий підбір
NON_PRIMARY_PENALTY = 2.0      # штраф за гру не на основній ролі (бали overall)

def _group_of(pos: str) -> str:
    # Мапимо позицію до групи
    if pos == "GK": return "GK"
    if pos in DEF:  return "DEF"
    if pos in MID:  return "MID"
    if pos in FWD:  return "FWD"
    return "OTHER"

def _eligible_for_group(positions_set: set, group_name: str) -> bool:
    # Перевіряємо придатність за будь-якою позицією у списку
    if group_name == "GK":  return "GK" in positions_set
    if group_name == "DEF": return len(positions_set & DEF) > 0
    if group_name == "MID": return len(positions_set & MID) > 0
    if group_name == "FWD": return len(positions_set & FWD) > 0
    return False

def best_starting_xi_score(club_df: pd.DataFrame):
    # Дві поведінки: жорстко за primary або гнучко за списком позицій
    sub = club_df.copy()
    sub = sub.drop_duplicates("sofifa_id")

    if not FLEXIBLE_POSITIONS or "player_positions" not in sub.columns:
        # Початкова логіка (жорстка)
        sub = sub.sort_values("overall", ascending=False)
        gk = sub[sub["primary_position"]=="GK"].head(1)
        defs = sub[sub["primary_position"].isin(DEF)].head(4)
        mids = sub[sub["primary_position"].isin(MID)].head(4)
        fwds = sub[sub["primary_position"].isin(FWD)].head(2)
        if (len(gk)<1) or (len(defs)<4) or (len(mids)<4) or (len(fwds)<2):
            return None
        xi = pd.concat([gk, defs, mids, fwds], axis=0)
        return {"avg_overall_xi": xi["overall"].mean(),
                "players_ids": list(xi["sofifa_id"].values),
                "n_selected": len(xi)}

    #  Гнучкий підбір через Hungarian
    # Слоти 1–4–4–2
    slots = (["GK"]*1) + (["DEF"]*4) + (["MID"]*4) + (["FWD"]*2)
    n_slots = len(slots)

    # Підготовка гравців
    sub = sub[pd.notna(sub["overall"])]
    if sub.empty:
        return None

    # Множина позицій з рядка player_positions
    pos_lists = sub["player_positions"].astype(str).str.split(",").apply(lambda xs: {x.strip() for x in xs})
    primary_groups = sub["primary_position"].astype(str).apply(_group_of)

    players = sub.reset_index(drop=True)
    players["positions_set"] = pos_lists
    players["primary_group"] = primary_groups

    # Перевірка достатності кандидатів за групами
    need = {"GK":1, "DEF":4, "MID":4, "FWD":2}
    have = {
        "GK":  (players["positions_set"].apply(lambda s: "GK" in s)).sum(),
        "DEF": (players["positions_set"].apply(lambda s: len(s & DEF)>0)).sum(),
        "MID": (players["positions_set"].apply(lambda s: len(s & MID)>0)).sum(),
        "FWD": (players["positions_set"].apply(lambda s: len(s & FWD)>0)).sum(),
    }
    if any(have[g] < need[g] for g in need):
        return None

    # Матриця оцінок: overall - штраф, якщо не первинна група
    n_players = len(players)
    scores = np.full((n_players, n_slots), -1e9, dtype=float)

    for i in range(n_players):
        ov = float(players.loc[i, "overall"])
        pset = players.loc[i, "positions_set"]
        pgrp = players.loc[i, "primary_group"]
        for j, grp in enumerate(slots):
            if _eligible_for_group(pset, grp):
                penalty = 0.0 if pgrp == grp else NON_PRIMARY_PENALTY
                scores[i, j] = ov - penalty

    # Hungarian (мінімізуємо вартість → беремо -scores)
    cost = -scores
    row_ind, col_ind = linear_sum_assignment(cost)

    # Обираємо тільки валідні призначення (де було не -1e9)
    chosen = [(r, c) for r, c in zip(row_ind, col_ind) if scores[r, c] > -1e8]
    if len(chosen) < n_slots:
        return None

    sel_idx = [r for r, _ in sorted(chosen, key=lambda x: x[1])]
    xi = players.iloc[sel_idx]

    return {"avg_overall_xi": xi["overall"].mean(),
            "players_ids": list(xi["sofifa_id"].values),
            "n_selected": len(xi)}

rows = []
for club, grp in club_df.groupby("club_name"):
    res = best_starting_xi_score(grp)
    if res: rows.append({"club_name": club, **res})

xi_table = pd.DataFrame(rows).sort_values("avg_overall_xi", ascending=False)
xi_table.head(30).to_csv(OUT/"clubs_best_starting_xi_top30.csv", index=False)


print("OK. Результати (1–7) збережено у out/tables/")
